In [1]:
import lib.utils.bookkeeping as bookkeeping
from config.config_train_sample import get_config
import torch
import torch.nn as nn
import ml_collections
import yaml
import lib.utils.bookkeeping as bookkeeping
from pathlib import Path
import torch.utils.tensorboard as tensorboard
from tqdm import tqdm
import sys
import signal
import argparse
from config.config_train_sample import get_config
import matplotlib.pyplot as plt
import ssl
import os 
ssl._create_default_https_context = ssl._create_unverified_context
import lib.models.models as models
import lib.models.model_utils as model_utils
import lib.datasets.datasets as datasets
import lib.datasets.dataset_utils as dataset_utils
import lib.losses.losses as losses
import lib.losses.losses_utils as losses_utils
import lib.training.training as training
import lib.training.training_utils as training_utils
import lib.optimizers.optimizers as optimizers
import lib.optimizers.optimizers_utils as optimizers_utils
import lib.loggers.loggers as loggers
import lib.loggers.logger_utils as logger_utils
import lib.sampling.sampling as sampling
import lib.sampling.sampling_utils as sampling_utils

2023-09-04 16:52:33.465142: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
cfig = get_config()

path = '/Users/paulheller/PythonRepositories/Master-Thesis/ContTimeDiscreteSpace/tauLDR/SavedModels/MNIST'
bookkeeping.save_config(cfig, cfig.save_location)
print(cfig.training.n_iters)

<class 'ml_collections.config_dict.config_dict.ConfigDict'>
5000


In [10]:

cfg = bookkeeping.load_config('/Users/paulheller/PythonRepositories/Master-Thesis/ContTimeDiscreteSpace/tauLDR/SavedModels/MNIST/2023-09-04/config_001.yaml')
print(cfg)

data:
  S: 256
  batch_size: 64
  download: true
  name: DiscreteMNIST
  random_flips: true
  root: /Users/paulheller/PythonRepositories/Master-Thesis/ContTimeDiscreteSpace/tauLDR/lib/datasets/MNIST
  shape: !!python/object/new:ruamel.yaml.comments.CommentedSeq
    listitems:
    - 1
    - 32
    - 32
    state:
      _yaml_format: !!python/object/new:ruamel.yaml.comments.Format
        state: !!python/tuple
        - null
        - _flow_style: false
      _yaml_line_col: !!python/object:ruamel.yaml.comments.LineCol
        col: 2
        data:
          0:
          - 9
          - 4
          1:
          - 10
          - 4
          2:
          - 11
          - 4
        line: 9
  shuffle: true
  train: true
device: cpu
distributed: false
experiment_name: mnist
init_model_path: null
loss:
  eps_ratio: !!python/object/new:ruamel.yaml.scalarfloat.ScalarFloat
    args:
    - 1.0e-09
    state:
      _e_sign: true
      _e_width: 3
      _exp: e
      _m_lead0: 0
      _m_sign: false


In [4]:
print(type(cfg))
print(cfg.training.n_iters)

<class 'ml_collections.config_dict.config_dict.ConfigDict'>
5000


In [13]:
device = torch.device(cfg.device)

model = model_utils.create_model(cfg, device)
print("number of parameters: ", sum([p.numel() for p in model.parameters()]))

loss = losses_utils.get_loss(cfg)

training_step = training_utils.get_train_step(cfg)

optimizer = optimizers_utils.get_optimizer(model.parameters(), cfg)
print(optimizer)

sampler = sampling_utils.get_sampler(cfg)

state = {"model": model, "optimizer": optimizer, "n_iter": 0}
print(state['n_iter'])
bookkeeping.save_state(state, cfg.save_location)

number of parameters:  1623458
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0002
    maximize: False
    weight_decay: 0
)
0


In [7]:
checkpoint_path = '/Users/paulheller/PythonRepositories/Master-Thesis/ContTimeDiscreteSpace/tauLDR/SavedModels/MNIST'
date = '2023-09-04'
model_name = 'ckpt_0000004999.pt'
checkpoint_path = os.path.join(checkpoint_path, date, model_name)
print(checkpoint_path)
#print('/Users/paulheller/PythonRepositories/Master-Thesis/ContTimeDiscreteSpace/tauLDR/SavedModels/MNIST/2023-09-04/ckpt_0000004999.pt')
state = bookkeeping.load_state(state, checkpoint_path)

print(state['n_iter'])

/Users/paulheller/PythonRepositories/Master-Thesis/ContTimeDiscreteSpace/tauLDR/SavedModels/MNIST/2023-09-04/ckpt_0000004999.pt
4999
